In [1]:
# Load library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
%matplotlib inline

## Import Data 

In [7]:
# Create URL
test_csv = "~/COMP30027_2021_Project2_datasets/recipe_test.csv"
train_csv = "~/COMP30027_2021_Project2_datasets/recipe_train.csv"

In [8]:
# Load Dataset 
test_df = pd.read_csv(test_csv )
train_df = pd.read_csv(train_csv)

In [31]:
# Load CountVec model
import pickle
name_vectorizer = pickle.load(open("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_countvec/train_name_countvectorizer.pkl", "rb"))
name_dict = name_vectorizer.vocabulary_

In [32]:
# Extract Features from name column using Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
name_list = train_df['name'].values # Create an array of name 
corpus = name_list
X = name_vectorizer.fit_transform(corpus)

In [34]:
print(X.shape)

(40000, 10892)
